In [5]:
"""
This script extracts camera extrinsics and intrinsics using COLMAP.

The directory structure of the root directory should be as follows:
root_dir
|   calib
|   |   *cam*
|   |   |   image_*.jpg

Run the script using:
    python colmap_params.py -r <root_dir>

The parameters will be stored in <root_dir>/calib/params.txt.
"""

import os
import logging
import subprocess
import sqlite3
import numpy as np
import cv2
import shutil
from numpy.lib import recfunctions as rf
from tqdm import tqdm
from cv2 import aruco
from glob import glob
from argparse import ArgumentParser
import moviepy.editor

logging.getLogger().setLevel(logging.INFO)

MARKER_AREA_THRESHOLD = 1000 # Parameter
MAX_IMAGE_ID = 2**31 - 1

debug = False

tmp_dir = os.path.join("tmp", "brics_calib")

def create_dir(path: str):
    """ Creates a directory also deleting previous one if it exissts """
    logging.warning(f"Deleting files at {path}")
    try:
        shutil.rmtree(path)
    except:
        pass
    os.makedirs(path)

def image_ids_to_pair_id(image_id1, image_id2):
    if image_id1 > image_id2:
        image_id1, image_id2 = image_id2, image_id1
    return image_id1 * MAX_IMAGE_ID + image_id2

def array_to_blob(array):
    return array.tobytes()

def blob_to_array(blob, dtype, shape=(-1,)):
    return np.frombuffer(blob, dtype=dtype).reshape(*shape)

def get_images(args, db_path, image_dir):
    connection = sqlite3.connect(db_path)
    cursor = connection.cursor()
    cursor.execute("SELECT image_id, name FROM images;")
    image_ids, image_paths = zip(*cursor.fetchall())
    image_ids = list(image_ids)
    image_paths = [os.path.join(image_dir, image_path) for image_path in image_paths]
    cursor.close()
    connection.close()
    return image_ids, image_paths

def add_features(image_ids, image_paths, db_path):
    dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_250)
    parameters =  cv2.aruco.DetectorParameters()
    detector = cv2.aruco.ArucoDetector(dictionary, parameters)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.0001) # Sub-pixel detection

    connection = sqlite3.connect(db_path)
    cursor = connection.cursor()
    cursor.execute("DELETE FROM keypoints;")
    cursor.execute("DELETE FROM descriptors;")
    cursor.execute("DELETE FROM matches;")
    connection.commit()

    logging.info("Extracting features from ChArUco")
    image_id_desc = {}
    for image_id, image_path in tqdm(list(zip(image_ids, image_paths))):
        frame = cv2.imread(image_path)

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        orig_corners, orig_ids, _ = detector.detectMarkers(gray)

        if len(orig_corners) <= 0:
            logging.warning("No markers found in {image_path}")
            continue
        else:
            corners = []
            ids = []
            for (corner, id) in zip(orig_corners, orig_ids):
                area = cv2.contourArea(corner)
                if area >= MARKER_AREA_THRESHOLD:  # PARAM
                    ids.append(id)
                    corners.append(corner)

            if len(orig_corners) - len(corners) > 0:
                logging.warning(f'Ignoring {len(orig_corners) - len(corners)} sliver markers.')

            ids = np.asarray(ids).flatten()
            # ipdb.set_trace()
            uniq, cnt = np.unique(ids, return_counts=True)
            if np.any(cnt > 1):
                if debug:
                    frame_markers = aruco.drawDetectedMarkers(frame.copy(), corners, ids)
                    cv2.imshow("Duplicate markers", frame_markers)
                    ch = cv2.waitKey(0)
                    if ch == 27:
                        exit()
                logging.warning(f"{np.count_nonzero(cnt > 1)} duplicate IDs found, ignoring")

            non_uniq = dict(zip(uniq, cnt))

            uniq_corners = []
            uniq_ids = []
            for i in range(len(corners)):
                # Skip if ID is non-unique
                if non_uniq[ids[i]] > 1:
                    continue

                if np.all(corners[i] >= 0):
                    cv2.cornerSubPix(gray, corners[i], winSize=(3, 3), zeroZone=(-1, -1), criteria=criteria)
                    corners[i] = corners[i].squeeze()
                    uniq_ids.append(ids[i])
                    uniq_corners.append(corners[i])
                else:
                    raise NotImplementedError

            ids = np.asarray(uniq_ids)
            uniq, cnt = np.unique(ids, return_counts=True)

            # Insert keypoints
            if len(uniq_corners) > 0:
                keypoints = np.concatenate(uniq_corners)
                cursor.execute(
                    "INSERT INTO keypoints VALUES (?, ?, ?, ?)",
                    (image_id,) + keypoints.shape + (array_to_blob(keypoints.astype(np.float32)),)
                )

                ids = np.repeat(ids, 4)
                for i in range(4):
                    # TODO: Find a more elegant solution for this
                    ids[i::4] += (i*1000)
                image_id_desc[image_id] = ids

        connection.commit()

    cursor.close()
    connection.close()
    return image_id_desc

def exhaustive_match(image_ids, image_paths, db_path, args, image_id_desc, image_dir):
    connection = sqlite3.connect(db_path)
    cursor = connection.cursor()
    cursor.execute("DELETE FROM matches;")
    connection.commit()

    logging.info("Matching features")
    image_pairs = []
    for i in tqdm(range(len(image_ids))):
        if image_ids[i] not in image_id_desc:
            continue

        for j in range(len(image_ids)):
            if (image_ids[i] >= image_ids[j]) or (image_ids[j] not in image_id_desc):
                continue

            desc1 = image_id_desc[image_ids[i]]
            desc2 = image_id_desc[image_ids[j]]

            # Find matches
            matches = []
            for k in range(desc1.shape[0]):
                for l in range(desc2.shape[0]):
                    if desc1[k] == desc2[l]:
                        matches.append([k, l])

            # Insert into database
            pair_id = image_ids_to_pair_id(image_ids[i], image_ids[j])
            if not matches:
                continue

            image_pairs.append([image_paths[i], image_paths[j]])
            matches = np.asarray(matches, np.uint32)

            cursor.execute("INSERT INTO matches VALUES (?, ?, ?, ?)",
                           (pair_id,) + matches.shape + (array_to_blob(matches),))

            connection.commit()

    cursor.close()
    connection.close()

    match_list_path = os.path.join(tmp_dir, "match_list.txt")
    logging.info(f"Writing image pairs at {match_list_path}")
    with open(os.path.join(match_list_path), "w") as f:
        for pair in image_pairs:
            f.write((" ".join(pair)).replace(image_dir + "/", "") + "\n")


In [6]:
time_window = np.linspace(60, 110, 100)

for cam_idx in range(2,5):
    fpath = f'videos/CalibrationCam{cam_idx}_03-08-1736_cam{cam_idx}.mp4'

    if os.path.exists(f'calib/cam_{cam_idx}'):
        files = os.listdir(f'calib/cam_{cam_idx}')
        for file in files:
            os.remove(f'calib/cam_{cam_idx}/{file}')
        os.rmdir(f'calib/cam_{cam_idx}')

    os.mkdir(f'calib/cam_{cam_idx}')
    clip = moviepy.editor.VideoFileClip(fpath)
    for image_idx, frame_time in enumerate(time_window):
        clip.save_frame(f'calib/cam_{cam_idx}/image_{image_idx}.jpg', t=frame_time)

In [7]:
parser = ArgumentParser()
parser.add_argument("-r", "--root_dir", required=True, help="Base directory")
parser.add_argument("--separate_calib", action="store_true")
args = parser.parse_args(['-r', '.'])

db_path = os.path.join(tmp_dir, "db.db")
if args.separate_calib:
    input_image_path = os.path.join(args.root_dir, "image", "synced")
else:
    input_image_path = os.path.join(args.root_dir, "calib")

image_path = os.path.join(tmp_dir, "images")
create_dir(image_path)

image_dirs = list(sorted(glob(f"{input_image_path}/*cam*")))

In [3]:

for image_dir in image_dirs:
    # to_copy = os.path.basename(sorted(glob(f"{image_dir}/*.jpg"))[0])
    # os.makedirs(os.path.join(image_path, os.path.basename(image_dir)))
    # shutil.copyfile(os.path.join(image_dir, to_copy),
    #                 os.path.join(image_path, os.path.basename(image_dir), to_copy))
    image_list = sorted(glob(f"{image_dir}/*.jpg"))
    os.makedirs(os.path.join(image_path, os.path.basename(image_dir)))
    for to_copy in image_list:
        to_copy = os.path.basename(to_copy)
        shutil.copyfile(os.path.join(image_dir, to_copy),
                        os.path.join(image_path, os.path.basename(image_dir), to_copy))

if os.path.exists(db_path):
    logging.warning("Previous database found, deleting.")
    os.remove(db_path)
    try:
        os.remove(os.path.join(tmp_dir, "db.db-wal"))
        os.remove(os.path.join(tmp_dir, "db.db-shm"))
    except FileNotFoundError:
        pass

logging.info("Importing images in database")
subprocess.run([
    "colmap", "feature_extractor",
    "--database_path", db_path,
    "--image_path", image_path,
    "--ImageReader.single_camera_per_folder", "1",
    "--ImageReader.camera_model", "OPENCV",
    "--SiftExtraction.use_gpu", "0"]
)

image_ids, image_paths = get_images(args, db_path, input_image_path)


INFO:root:Importing images in database



Feature extraction

Processed file [1/1500]
  Name:            cam_2/image_100.jpg
  Dimensions:      1440 x 1056
  Camera:          #1 - OPENCV
  Focal Length:    1728.00px
  Features:        4789
Processed file [2/1500]
  Name:            cam_2/image_103.jpg
  Dimensions:      1440 x 1056
  Camera:          #1 - OPENCV
  Focal Length:    1728.00px
  Features:        4654
Processed file [3/1500]
  Name:            cam_2/image_101.jpg
  Dimensions:      1440 x 1056
  Camera:          #1 - OPENCV
  Focal Length:    1728.00px
  Features:        4785
Processed file [4/1500]
  Name:            cam_2/image_10.jpg
  Dimensions:      1440 x 1056
  Camera:          #1 - OPENCV
  Focal Length:    1728.00px
  Features:        3927
Processed file [5/1500]
  Name:            cam_2/image_102.jpg
  Dimensions:      1440 x 1056
  Camera:          #1 - OPENCV
  Focal Length:    1728.00px
  Features:        4700
Processed file [6/1500]
  Name:            cam_2/image_0.jpg
  Dimensions:      1440 x 105

In [12]:
image_id_desc = add_features(image_ids, image_paths, db_path)


INFO:root:Extracting features from ChArUco
100%|██████████| 1500/1500 [00:47<00:00, 31.42it/s]


In [13]:
exhaustive_match(image_ids, image_paths, db_path, args, image_id_desc, input_image_path)

INFO:root:Matching features
100%|██████████| 1500/1500 [33:38<00:00,  1.35s/it] 
INFO:root:Writing image pairs at tmp/brics_calib/match_list.txt


In [4]:


logging.info("Performing geometric verification")
subprocess.run([
    "colmap", "matches_importer",
    "--database_path", db_path,
    "--match_list_path", f"{os.path.join(tmp_dir, 'match_list.txt')}",
    "--match_type", "pairs",
    "--SiftMatching.min_num_inliers", "1",
    "--SiftMatching.use_gpu", "0"
])




INFO:root:Performing geometric verification



Custom feature matching

Matching block [1/5934]

*** Aborted at 1675724944 (unix time) try "date -d @1675724944" if you are using GNU date ***
PC: @     0x55a2855822f8 (unknown)
*** SIGSEGV (@0x0) received by PID 39791 (TID 0x7f952aff5700) from PID 0; stack trace: ***
    @     0x7f953ce75631 (unknown)
    @     0x7f953c3b8420 (unknown)
    @     0x55a2855822f8 (unknown)
    @     0x55a285584470 colmap::TwoViewGeometry::EstimateUncalibrated()
    @     0x55a28559d1df colmap::TwoViewGeometryVerifier::Run()
    @     0x55a2856a5101 colmap::Thread::RunFunc()
    @     0x7f953b6bade4 (unknown)
    @     0x7f953c3ac609 start_thread
    @     0x7f953b363133 clone


CompletedProcess(args=['colmap', 'matches_importer', '--database_path', 'tmp/brics_calib/db.db', '--match_list_path', 'tmp/brics_calib/match_list.txt', '--match_type', 'pairs', '--SiftMatching.min_num_inliers', '1', '--SiftMatching.use_gpu', '0'], returncode=-11)

In [16]:
logging.info("Reconstructing")
recon_path = os.path.join(tmp_dir, "reconstruction")
create_dir(recon_path)
subprocess.run([
    "colmap", "mapper",
    "--database_path", db_path,
    "--image_path", image_path,
    "--output_path", recon_path,
])

INFO:root:Reconstructing



Loading database

Loading cameras... 3 in 0.000s
Loading matches... 0 in 0.000s
Loading images... 1500 in 0.002s (connected 0)
Building correspondence graph... in 0.000s (ignored 0)

Elapsed time: 0.000 [minutes]




CompletedProcess(args=['colmap', 'mapper', '--database_path', 'tmp/brics_calib/db.db', '--image_path', 'tmp/brics_calib/images', '--output_path', 'tmp/brics_calib/reconstruction'], returncode=0)

In [33]:
logging.info("Performing global bundle adjustment")
subprocess.run([
    "colmap", "bundle_adjuster",
    "--input_path", os.path.join(recon_path, "0"),
    "--output_path", os.path.join(recon_path, "0"),
    "--BundleAdjustment.refine_principal_poin", "1"
])

INFO:root:Performing global bundle adjustment



Global bundle adjustment

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  5.566408e+02    0.00e+00    5.19e-02   0.00e+00   0.00e+00  1.00e+04        0    2.14e-02    2.57e-02
   1  5.566397e+02    1.18e-03    1.39e+00   1.76e-01   1.00e+00  3.00e+04        1    5.94e-02    8.54e-02
   2  5.566362e+02    3.44e-03    1.73e+00   4.50e-01   1.00e+00  9.00e+04        1    7.10e-02    1.57e-01
   3  5.566263e+02    9.91e-03    1.03e+01   1.50e+00   1.00e+00  2.70e+05        1    1.00e-01    2.57e-01
   4  5.565997e+02    2.66e-02    7.64e+01   5.04e+00   9.83e-01  8.10e+05        1    6.41e-02    3.22e-01
   5  5.565629e+02    3.68e-02    6.03e+02   1.41e+01   5.50e-01  8.11e+05        1    7.09e-02    3.93e-01
   6  5.564975e+02    6.54e-02    5.64e+02   1.26e+01   7.35e-01  9.05e+05        1    9.04e-02    4.84e-01
   7  5.564430e+02    5.45e-02    6.58e+02   1.31e+01   6.41e-01  9.26e+05        1    7.60e-02    5.60e-01
 

CompletedProcess(args=['colmap', 'bundle_adjuster', '--input_path', 'tmp/brics_calib/reconstruction/0', '--output_path', 'tmp/brics_calib/reconstruction/0', '--BundleAdjustment.refine_principal_poin', '1'], returncode=0)

In [34]:
output_dir = os.path.join(tmp_dir, "output")
create_dir(output_dir)
subprocess.run([
    "colmap", "model_converter",
    "--input_path", os.path.join(recon_path, "0"),
    "--output_path", output_dir,
    "--output_type", "TXT"
])

CompletedProcess(args=['colmap', 'model_converter', '--input_path', 'tmp/brics_calib/reconstruction/0', '--output_path', 'tmp/brics_calib/output', '--output_type', 'TXT'], returncode=0)

In [36]:
# Read images
image_params = []
with open(os.path.join(output_dir, "images.txt")) as f:
    skip_next = False
    for line in f.readlines():
        if skip_next:
            skip_next = False
            continue
        if line.startswith("#"):
            continue
        data = line.split()
        param = []
        param.append(int(data[8]))
        param.append(data[9].split("/")[0])
        param += [float(datum) for datum in data[1:8]]
        image_params.append(tuple(param))
        skip_next = True

images = np.array(image_params, dtype=[
    ('cam_id', int), ('cam_name', '<U22'),
    ('qvecw', float), ('qvecx', float), ('qvecy', float), ('qvecz', float),
    ('tvecx', float), ('tvecy', float), ('tvecz', float)
])

# Read cameras
cam_params = []
with open(os.path.join(output_dir, "cameras.txt")) as f:
    for line in f.readlines():
        if line.startswith("#"):
            continue
        data = line.split()
        param = []
        param.append(int(data[0]))
        param.append(int(data[2]))
        param.append(int(data[3]))
        param += [float(datum) for datum in data[4:]]
        cam_params.append(tuple(param))
cameras = np.array(cam_params, dtype=[
    ('cam_id', int),
    ('width', int), ('height', int),
    ('fx', float), ('fy', float),
    ('cx', float), ('cy', float),
    ('k1', float), ('k2', float),
    ('p1', float), ('p2', float),
])

img_cams = rf.join_by('cam_id', cameras, images)
if args.separate_calib:
    create_dir(os.path.join(args.root_dir, 'calib'))
np.savetxt(os.path.join(args.root_dir, 'calib', 'params.txt'), img_cams, fmt="%s", header=" ".join(img_cams.dtype.fields))

logging.warning(f"Stored the parameters at {os.path.join(args.root_dir, 'calib', 'params.txt')}")